# Redes neuronales recurrentes

## Caso de estudio: Generación de texto

### Datos de tipo texto y redes neuronales
Recordemos que todas las entradas en una red neuronal
deben ser tensores de datos numéricos. Cualquier dato que se necesite
procesar. Primero debe ser convertido en un tensor
numérico, un paso llamado "vectorización" de datos.

Para ello usaremos *Word Embedding*, crea vectores de tamaños reducidos pero que preservan las relaciones semánticas. Un detalle muy importante.

### *Character-Level Language Models*
Consiste en darle a la RNN una
palabra y se le pide que modele la distribución de probabilidad del siguiente
carácter que le correspondería a la secuencia de caracteres anteriores. Con
este modelo, si lo llamamos repetitivamente, podremos generar texto carácter
a carácter.

Para usar el modelo, introducimos un carácter en la RNN y obtenemos una
distribución sobre qué carácter probablemente será el siguiente. Tomamos
una muestra de esta distribución y la retroalimentamos para obtener el
siguiente carácter. ¡Repetimos este proceso y estamos generando texto!

## Descarga y preprocesado de datos
El primer paso en este ejemplo será el descargar y preparar el conjunto de
datos con el que entrenaremos nuestra red neuronal:

In [ ]:
import tensorflow as tf
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1a.txt',
'https://raw.githubusercontent.com/jorditorresBCN/Deep-Learning-Introduccion-practica-con-Keras/master/DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto: {} carácteres'.format(len(text)))
vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

212992/203286 [===============================] - 0s 0us/step
Longitud del texto: 203251 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


Como estas tratando el caso de estudio a nivel de carácter, podríamos considerar que aquí el corpus (recopilatorio) son los caracteres, por tanto un corpus muy pequeño. 

Recordemos que las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos "tablas de traducción": una de caracteres a números y otra de números a caracteres:

In [ ]:
import numpy as np

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Ahora tenemos una representacion de un numero para cada caracter
for char,_ in zip(char2idx, range(len(vocab))):
  print(' {:4s}: {:3d},'.format(repr(char), char2idx[char]))

 '\n':   0,
 '\r':   1,
 ' ' :   2,
 '!' :   3,
 '"' :   4,
 '#' :   5,
 '%' :   6,
 "'" :   7,
 '(' :   8,
 ')' :   9,
 '*' :  10,
 '+' :  11,
 ',' :  12,
 '-' :  13,
 '.' :  14,
 '/' :  15,
 '0' :  16,
 '1' :  17,
 '2' :  18,
 '3' :  19,
 '4' :  20,
 '5' :  21,
 '6' :  22,
 '7' :  23,
 '8' :  24,
 '9' :  25,
 ':' :  26,
 ';' :  27,
 '<' :  28,
 '=' :  29,
 '>' :  30,
 '?' :  31,
 '@' :  32,
 'A' :  33,
 'B' :  34,
 'C' :  35,
 'D' :  36,
 'E' :  37,
 'F' :  38,
 'G' :  39,
 'H' :  40,
 'I' :  41,
 'J' :  42,
 'K' :  43,
 'L' :  44,
 'M' :  45,
 'N' :  46,
 'O' :  47,
 'P' :  48,
 'Q' :  49,
 'R' :  50,
 'S' :  51,
 'T' :  52,
 'U' :  53,
 'V' :  54,
 'W' :  55,
 'X' :  56,
 'Y' :  57,
 '[' :  58,
 ']' :  59,
 '_' :  60,
 'a' :  61,
 'b' :  62,
 'c' :  63,
 'd' :  64,
 'e' :  65,
 'f' :  66,
 'g' :  67,
 'h' :  68,
 'i' :  69,
 'j' :  70,
 'k' :  71,
 'l' :  72,
 'm' :  73,
 'n' :  74,
 'o' :  75,
 'p' :  76,
 'q' :  77,
 'r' :  78,
 's' :  79,
 't' :  80,
 'u' :  81,
 'v' :  82,
 'w'

Ahora tenemos una representación de entero (integer) para cada carácter. Y ahora esta función inversa a la anterior podemos pasar el texto a enteros:

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


## Preparación de los datos para ser usados por la RNN
Para entrenar el modelo prepararemos unas secuencias de caracteres como
entradas y salida de un tamaño determinado. En nuestro ejemplo hemos
definido el tamaño de 100 caracteres con la variable seq_length.

Empezamos dividiendo el texto que tenemos en secuencias de
seq_length+1 de caracteres con las cuales luego contruiremos los datos de
entrenamiento compuestos por las entradas de seq_length caracteres y las
salidas correspondientes que contienen la misma longitud de texto, excepto
que se desplaza un carácter a la derecha.

Usaremos la función tf.data.Dataset.from_tensor_slices, que
crea un conjunto de datos con el contenido del tensor text_as_int que
contiene el texto, al que podremos aplicar el método batch( )para dividir
este conjunto de datos en secuencias de seq_length+1 de índice de caracteres:

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Podemos comprobar que *sequences* contiene el texto dividido en paquetes
de 101 caracteres como esperamos (por ejemplo mostremos las 10 primeras
secuencias):

In [ ]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

De esta secuencia se obtiene el conjunto de datos de training que contenga
tanto los datos de entrada (desde la posición 0 a la 99) como los datos de
salida (desde la posición 1 a la 100). Para ello se crea una función que realiza
esta tarea y se aplica a todas las secuencias usando el método *map( )* de la
siguiente forma:

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
  
dataset = sequences.map(split_input_target)

En este punto, dataset contiene un conjunto de parejas de secuencias de
texto (con la representación numérica de los caracteres), donde el primer componente de la pareja contiene un paquete con una secuencia de 100
caracteres del texto original y la segunda su correspondiente salida, también
de 100 caracteres. Podemos comprobarlo visualizándolo por pantalla (por
ejemplo mostrando la primera pareja):

In [ ]:
for input_example, target_example in dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data: 'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


En este punto del código disponemos de los datos de entrenamiento en el
tensor dataset en forma de parejas de secuencias de 100 integers de 64 bits
que representan un carácter del vocabulario:

In [ ]:
print(dataset)

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


En realidad los datos ya están preprocesados en el formato que se requiere
para ser usados en el entreno de la red neuronal, pero recordemos que en
redes neuronales los datos se agrupan en batches antes de pasarlos al modelo.
En nuestro caso hemos decidido un tamaño de batch de 64, que nos facilita la
explicación, este es un
hiperparámetro importante de ajustar correctamente teniendo en cuenta
diferentes factores, como el tamaño de la memoria disponible, por poner un
ejemplo. En este código, para crear los batches de parejas de secuencias hemos
considerado usar tf.data que además nos permite barajar84 las secuencias
previamente:

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 64, 100), dtype=tf.int64, name=None))>


Recapitulando, ahora en el tensor dataset disponemos de los datos de
entrenamiento ya listos para ser usados para entrenar el modelo: batches
compuestos de 64 parejas de secuencias de 100 integers de 64 bits que
representan el carácter correspondiente en el vocabulario.

## Construcción del modelo RNN
Para construir el modelo usaremos tf.keras.Sequential que ya
conocemos. Usaremos una versión mínima de RNN para facilitar la
explicación, que contenga solo una capa LSTM. En concreto definimos una
red de solo 3 capas:

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model



vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

La primera capa es de tipo Word Embedding como las que antes hemos
presentado muy brevemente que mapea cada carácter de entrada en un
vector Embedding. Esta capa tf.keras.layers. Embedding permite
especificar varios argumentos que se pueden consultar en todo detalle en el
manual de TensorFlow.

En nuestro caso el primero que especificamos es el tamaño del vocabulario,
indicado con el argumento vocab_size, que indica cuantos vectores
Embedding tendrá la capa. A continuación indicamos las dimensiones de estos
vectores Embedding mediante el argumento embedding_dim, que en nuestro
caso hemos decidido que sea 256. Finalmente se indica el tamaño del batch
que usaremos para entrenar, en nuestro caso 64.

La segunda capa es de tipo LSTM introducida anteriormente en este
capítulo. Esta capa tf.keras.layers.LSTM tiene varios argumentos
posibles que se pueden consultar en el manual de TensorFlow, aquí solo
usaremos algunos y dejamos los valores por defecto del resto. Quizás el más
importante es el número de neuronas recurrentes que se indica con el
argumento units y que en nuestro caso hemos decidido que sea 1024
neuronas.

Con return_sequence se indica que queremos predecir el carácter siguiente a todos los caracteres de entrada, no solo el siguiente al último
carácter.

El argumento stateful indica, explicado de manera simple, el uso de las
capacidades de memoria de la red entre batches. Si este argumento está
instanciado a false se indica que a cada nuevo batch se inicializan las memory
cells comentadas anteriormente, mientras que si está a true se está indicando
para cada batch se mantendrán las actualizaciones hechas durante la
ejecución del bach anterior.

El último argumento que usamos es recurrent_kernel, donde indicamos
cómo se deben inicializar los pesos de las matrices internas de la red. En este
caso usamos la distribución uniforme glorot_uniform, habitual en estos
casos.

Finalmente la última capa es de tipo Dense, ya explicada previamente en
este libro. Aquí es importante el argumento units que nos dice cuantas
neuronas tendrá la capa y que nos marcará la dimensión de la salida. En
nuestro caso será igual al tamaño de nuestro vocabulario (vocab_size).

Como siempre, es interesante usar el método summary() para visualizar la
estructura del modelo:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           23552     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 92)            94300     
                                                                 
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


Podemos comprobar que la capa LSTM consta de muchos parámetros (más
de 5 millones) como era de esperar. Intentemos analizar un poco más esta
red neuronal. Para cada carácter de entrada (transformado a su equivalente
numérico), el modelo busca su vector de Embedding correspondiente y luego
ejecuta la capa LSTM con este vector Embedding como entrada. A la salida
de la LSTM aplica la capa Dense para decidir cual es el siguiente carácter.

Inspeccionemos las dimensiones de los tensores para poder comprender más
a fondo el modelo. Fijemonos en el primer batch del conjunto de datos de
entrenamiento y observemos su forma:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


Vemos que en esta red la secuencia de entrada son batch de 100 caracteres,
pero el modelo una vez entrenado puede ser ejecutado con cualquier tamaño
de cadena de entrada. Este es un detalle al que luego volveremos.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 92) # (batch_size, sequence_length, vocab_size)


la capa densa de
esta red neuronal no tiene una función de activación softmax como la capa
densa que se presentó en ejercicios anteriores. De aquí que retorne el vector con un
indicador de “evidencia” para cada carácter.

El siguiente paso es elegir uno de los caracteres. Sin entrar en detalle, no se eligirá el carácter más “probable” (mediante argmax) como se hizo en ejercicios anteriores puesto que el modelo pueda entrar en un bucle. Lo que se hará es
obtener una muestra de la distribución de salida. Pruébelo para el primer
ejemplo en el batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()
print (sampled_indices_characters)

[48 86 46 17 47 35 87  5 54 74 11 39 76 41 33  7 70 42 62 41 15 66 18 81
 83 43 70 60 17 40 81 48 12 26 18 87 46 82 21 14 35  4 41 52 48  6 89 70
 66 19  0 44 41 70  4 51 49 25 54 63 75 37  0 57 22 14 88 43 14 62 46 50
 59 40 22 48  0 83  9 21 60  5 84 66 79  2 62 91 43  9 85 61 21 69 69  6
 72 11  5 22]


Con tf.random.categorical se obtiene una muestra de una distribución
categórica y con squeeze se elimina la dimensiones del tensor de tamaño 1.
De esta manera en cada instante de tiempo se obtiene una predicción del
índice del siguiente carácter.

### Entrenamiento del modelo
En este punto, el problema puede tratarse como un problema de clasificación estándar para el que debemos definir la función de *Loss* y el optimizador.

Para
la
función
de
Loss
usaremos
la
función
estándar
tf.keras.losses.sparse_categorical_crossentropy dado que
estamos considerando datos categóricos. Dado que el retorno hemos visto
que se trata de unos valores de verisimilitud (no de probabilidades como si
hubiéramos ya aplicado softmax) se instanciará el argumento from_logits a
True



In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits,
                                                        from_logits=True)

En cuanto al optimizador usaremos tf.keras.optimizers.Adam con
los argumentos por defecto del optimizador Adam.

Con esta función de loss definida y usando el optimizador Adam con sus
argumentos por defecto, ya podemos llamar al método compile () de la
siguiente manera:

In [ ]:
model.compile(optimizer='adam', loss=loss)

En este ejemplo aprovecharemos para usar los Checkpoints87, una técnica de
tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La
idea es guardar una instantánea del estado del sistema periódicamente para
recuperar desde ese punto la ejecución en caso de fallo del sistema. En
nuestro caso, cuando entrenamos modelos Deep Learning, el Checkpoint lo
forman básicamente los pesos del modelo. Estos Checkpoint se pueden usar
también para hacer predicciones tal cual como haremos en este ejemplo.
La librería de Keras proporciona Checkpoints a través de la API Callbacks.
Concretamente
usaremos
tf.keras.callbacks.ModelCheckpoint88
para especificar cómo salvar los Checkpoints a cada epoch durante el
entrenamiento, a través de un argumento en el método fit() del modelo.

En el código debemos especificar el directorio en el que se guardarán los
Checkpoints que salvaremos y el nombre del fichero (que le añadiremos el número de epoch para nuestra comodidad):

In [ ]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Ahora ya está todo preparado para empezar a entrenar la red con el método
fit():

In [ ]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
31/31 [==============================] - 8s 175ms/step - loss: 3.2353
Epoch 2/50
31/31 [==============================] - 6s 173ms/step - loss: 2.7716
Epoch 3/50
31/31 [==============================] - 6s 175ms/step - loss: 2.4370
Epoch 4/50
31/31 [==============================] - 6s 175ms/step - loss: 2.2158
Epoch 5/50
31/31 [==============================] - 6s 173ms/step - loss: 2.0992
Epoch 6/50
31/31 [==============================] - 6s 175ms/step - loss: 1.9865
Epoch 7/50
31/31 [==============================] - 6s 175ms/step - loss: 1.8708
Epoch 8/50
31/31 [==============================] - 6s 174ms/step - loss: 1.7563
Epoch 9/50
31/31 [==============================] - 6s 173ms/step - loss: 1.6489
Epoch 10/50
31/31 [==============================] - 6s 174ms/step - loss: 1.5503
Epoch 11/50
31/31 [==============================] - 6s 175ms/step - loss: 1.4596
Epoch 12/50
31/31 [==============================] - 6s 174ms/step - loss: 1.3845
Epoch 13/50
31/31 [======

## Generación de texto usando el modelo RNN

Ahora que tenemos ya entrenado el modelo pasemos a usarlo para generar
texto. Para mantener este paso de predicción simple, vamos a usar un tamaño
de batch de 1. Debido a la forma en que se pasa el estado de la RNN de un
instante de tiempo al siguiente, el modelo solo acepta un tamaño de batch fijo
una vez construido. Por ello, para poder ejecutar el modelo con un tamaño
de batch diferente, necesitamos reconstruir manualmente el modelo con el
método build( ) del modelo y restaurar sus pesos desde el Checkpoints
(cogemos el ultimo con tf.train.latest_checkpoint ()):

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Ahora que tenemos el modelo entrenado y preparado para usar,
generaremos texto a partir de una palabra de partida con el siguiente código:

In [ ]:
def generate_text(model, start_string):
  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

El código empieza con inicializaciones como: definir el número de caracteres
a predecir con la variable num_generate, convertir la palabra inicial
(start_string) a su correspondiente representación numérica y preparan
lo tensores necesarios. Después se usa una variable temperature para decidir cómo de conservador en sus
predicciones queremos que se comporte nuestro modelo. En nuestro ejemplo
la hemos inicializado a 0.5. Con “temperaturas altas” (hasta 1) se permitirá más creatividad al modelo
para generar texto pero a costa de más errores (por ejemplo, errores
ortográficos, etc.). Mientras que con “temperaturas bajas” habrá menos
errores pero el modelo mostrará poca creatividad.

Luego, se usa una distribución categórica para calcular el índice del carácter
predicho(predictions y predicted_id).

Este carácter acabado de predecir se usa como nuestra próxima entrada al
modelo, retroalimentando el modelo para que ahora tenga más contexto (en
lugar de una sola letra). Después de predecir la siguiente letra, se
retroalimenta nuevamente, y así sucesivamente de manera que es cómo
aprende a medida que se obtiene más contexto de los carácteres predichos
previamente (input_val y text_generated.append())

Empecemos con una palabra que no conoce el corpus, por ejemplo
“Alcohol”, que nada tiene que ver con Deep Learning:

In [ ]:
print(generate_text(model, start_string="Alcohol "))

Alcohol en Keras se considera como una secuencia de capas que cada una de ellas. Si el lector al acabar el libro completo tenga una vision general del metodo fit() en la segunda parte del libro hablaremos mas sobre las diversas redes capitulos para los convolucionales para reducir los tamanos son raramente utilizados e probabilidad sobre clases de salida de tamanos obtener de mas detalle con siguientes de activacion ReLU. En este caso, estamos hablando de casi 100 millones de predicciones por segundo qu


Probemos ahora con una palabra como "modelo" o “activación” a ver que
pasa:

In [ ]:
print(generate_text(model, start_string="modelo "))

modelo de programacion, sin adornos y maximizando la legra san capas ocultas (agroyan en los casos anteriores, el codigo se puede requerir mas capacidad de computacion que antes solo estaba disponible para grandes organizaciones o gobiernos.
Acempara con el metodo summary() que indica que se anada con este ejemplo usaremos una conocida como funcion sigmoid102 que retorna el imagina el numero de personas que trabajan ya en Deep Learning y empresas que invierte que es una aproximacion que funciona muy b


In [ ]:
print(generate_text(model, start_string="activacion"))

activacion, ahora ya no son encontramos con redes neuronales. La ventaja de tanh es que esta indicando que el modelo que hemos creado en el caso de las redes neuronales, pensemos en conciso por informacion contenida en el conjunto de entrenamiento no es suficiente para entrenar a todas las neuronas de este primer argumento para aquellas esta concepto de peso W de la misma dimension que la entrada  y para ello softmax usa el valor de la funcion de activacion de la primera capa, pasando de una sigmoid a una


En resumen, el modelo presentado parece que ha aprendido a generar texto
de manera interesante, teniendo en cuenta el reducido dataset inicial con el
que se ha entrenado. Como ya hemos avanzado, proponemos que el lector
pruebe con otros conjuntos de datos de tipo texto. Por ejemplo en el artículo
“The Unreasonable Effectiveness of Recurrent Neural Network” del blog de Andrey
Karpathy91 el lector puede encontrar varios ejemplos de datos de tipo texto
que el lector puede usar directamente simplemente cambiando la URL del
fichero de texto de entrada al código propuesto en este capítulo

# Probando otro ejemplo

In [ ]:
import tensorflow as tf
import numpy as np

# DESCARGAMOS EL MODELO DE ENTRENAMIENTO
path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt',
'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))
print('Longitud del texto: {} carácteres'.format(len(text)))
print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))

#Creamos el conversor numero character index y viceversa
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Ahora tenemos un representacion de un caracter para cada numero
text_as_int = np.array([char2idx[c] for c in text])


Longitud del texto: 99993 carácteres
El texto está compuesto de estos 62 carácteres:


In [ ]:
# PREPARACION DE DATOS DE ENTREDA
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
# CONJUNTO DE DATOS ENTRADA/SALIDA
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
  
dataset = sequences.map(split_input_target)

In [ ]:
# CREACION DE BATCHES
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# DEFINICION DEL MODELO
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model



vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
# MOSTRAMOS LA INFORMACION DEL MODELO
model.summary()

print('----------------------------------------------------------------')
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

print('----------------------------------------------------------------')
for input_example_batch, target_example_batch in dataset.take(1):  
  example_batch_predictions = model(input_example_batch)
  print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           15872     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 62)            63550     
                                                                 
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________
----------------------------------------------------------------
Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)
----------------------------------------------------------------
Prediction:  (64, 100, 62) # (batch_size, sequence_length, vocab_size)


In [ ]:
# DEFINICION DE LOSS
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits,
                                                        from_logits=True)

In [ ]:
# COMPILAR EL MODELO
model.compile(optimizer='adam', loss=loss)

In [ ]:
# ESPECIFICAR CHECKPOINT
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# ENTRENAMIENTO DEL MODELO
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
15/15 [==============================] - 5s 185ms/step - loss: 3.5722
Epoch 2/50
15/15 [==============================] - 3s 179ms/step - loss: 3.2211
Epoch 3/50
15/15 [==============================] - 3s 190ms/step - loss: 3.1131
Epoch 4/50
15/15 [==============================] - 3s 180ms/step - loss: 2.8447
Epoch 5/50
15/15 [==============================] - 3s 178ms/step - loss: 2.5670
Epoch 6/50
15/15 [==============================] - 3s 185ms/step - loss: 2.4330
Epoch 7/50
15/15 [==============================] - 3s 180ms/step - loss: 2.3493
Epoch 8/50
15/15 [==============================] - 3s 175ms/step - loss: 2.2849
Epoch 9/50
15/15 [==============================] - 3s 185ms/step - loss: 2.2304
Epoch 10/50
15/15 [==============================] - 3s 179ms/step - loss: 2.1761
Epoch 11/50
15/15 [==============================] - 3s 181ms/step - loss: 2.1288
Epoch 12/50
15/15 [==============================] - 3s 180ms/step - loss: 2.0756
Epoch 13/50
15/15 [======

In [ ]:
# CARGA DEL MODELO
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
# FUNCION GENERADORA DE TEXTO
def generate_text(model, start_string):
  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# GENERACION DE TEXTO
print(generate_text(model, start_string="I "))

I should devery prace
As I did wear ere and give thee full of this exematake them for a king,
And now thee but seams and some I would not beng to contemply,
I think what thou hast not make her presss'd out of this to be of it,
I seak him bas with your awn backind the bloody offickings; and be it a lately and this beard not celsen about man;
And so she shall prieve your onf lords, a crotest be so,
And that I have make him coming; and we shall be faliand did in the sight of dongors lecking
The lows 
